In [1]:
import random
import numpy as np
from simulations import simulate_bids, simulate_regrets
import matplotlib.pyplot as plt

# Set the seed
random.seed(42)
np.random.seed(63)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


# Generate bids

In [2]:
norm_bids_dict = simulate_bids(
    dist_name="truncnorm",
    lower=1,
    upper=10,
    max_N=1000,
    max_n=100,
    max_N_train=100,
    max_n_train=500,
)
expon_bids_dict = simulate_bids(
    dist_name="truncexpon",
    lower=1,
    upper=10,
    max_N=1000,
    max_n=100,
    max_N_train=100,
    max_n_train=500,
)
pareto_bids_dict = simulate_bids(
    dist_name="truncpareto",
    lower=1,
    upper=10,
    max_N=1000,
    max_n=100,
    max_N_train=100,
    max_n_train=500,
)

# Run auctions

In [3]:
dist_names = ["truncnorm", "truncexpon", "truncpareto"]
bids_dicts = [norm_bids_dict, expon_bids_dict, pareto_bids_dict]
ns = [10, 20, 50, 100]
N_trains = [50, 100]
n_trains = [100, 200, 500]

In [4]:
regrets_dict = {}
for i, dist_name in enumerate(dist_names):
    regrets_to_plot = []
    for n in ns:
        regrets_to_plot.append(
            np.hstack(simulate_regrets(bids_dicts[i], "ecdf", N=1000, n=n))
        )
        regrets_to_plot.append(
            np.hstack(simulate_regrets(bids_dicts[i], "kde", N=1000, n=n))
        )
        for N_train in N_trains:
            for n_train in n_trains:
                regrets_to_plot.append(
                    np.hstack(
                        simulate_regrets(
                            bids_dicts[i],
                            "rde",
                            N=1000,
                            n=n,
                            N_train=N_train,
                            n_train=n_train,
                        )
                    )
                )
    regrets_dict[dist_name] = regrets_to_plot

In [8]:
# plt.boxplot(regrets_dict["truncnorm"])
len(regrets_dict["truncnorm"])

32

In [6]:
import pickle

# Write object to a file
with open("regrets.pkl", "wb") as file:
    pickle.dump(regrets_dict, file)